In [1]:
from openai import OpenAI
from pathlib import Path

key = open('openai.key').read().strip()
client = OpenAI(api_key=key, organization='org-l0QXnTWrsY221IPu8QIF1k1H', project='proj_hdXOnDa32fjKeT330EowFZsc')

In [2]:
pdf_path = Path('public_agenda_june24_.pdf')

In [3]:
message_file = client.files.create(file=pdf_path, purpose='assistants')

In [4]:
with open('prompt.txt') as f:
    prompt = f.read()

In [5]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
            "attachments": [
                { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
            ],
        }
    ]
)

In [6]:
assistant = client.beta.assistants.create(
    name="Technology Purchases",
    instructions=prompt,
    model='gpt-4o',
    tools=[{"type": "file_search"}],
)

In [7]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [8]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

{
  "technology_purchases": [
    {
      "technology_system_name": "Versadex Records Management System",
      "developer_company_name": "Versaterm Public Safety Inc.",
      "deploying_company_name": "Versaterm Public Safety Inc.",
      "budget": 4445367,
      "start_date": "2024-11-05",
      "end_date": "2028-11-04"
    }
  ]
}

